# Helper Functions

In [1]:
import numpy as np
import os
from PIL import Image
import Sokoban
import random

def numpyToString(array):
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))
    intArray = np.argmax(array, axis=0)
    output=""
    for (i,j), index in np.ndenumerate(intArray):
        if i > 0 and j == 0:
            output += "\n"
        output += int_to_char[index]
    return output

def stringToNumpy(string):
    data = np.array([list(l) for l in string.split("\n")])
    
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))

    encodingData = np.zeros((len(gameCharacters), data.shape[0], data.shape[1]))
    for (i,j),c in np.ndenumerate(data):
        index=char_to_int[c]
        encodingData[index][i][j]=1
    return encodingData

def numpyToImage(array, tileSize=16):
    width = array.shape[2]
    height = array.shape[1]
    lvlRows = numpyToString(array).split("\n")
    graphics = {
        "@":"assets/player.png",
        "#":"assets/wall.png",
        "$":"assets/box.png",
        "*":"assets/box_target.png",
        ".":"assets/target.png",
        "+":"assets/player_target.png"
    }
    for c in graphics:
        graphics[c] = Image.open(graphics[c]).convert('RGBA')
    graphics[" "] = Image.new("RGBA", (16,16), (0,0,0,0))
    lvlImage = Image.new("RGBA", (width*tileSize, height*tileSize), (100,212,113,255))
    grid = Image.open("assets/grid.png").convert('RGBA')
    for y,r in enumerate(lvlRows):
        for x,c in enumerate(r):
            lvlImage.paste(grid, (x*tileSize, y*tileSize, (x+1)*tileSize, (y+1)*tileSize), grid)
            lvlImage.paste(graphics[c], (x*tileSize, y*tileSize, (x+1)*tileSize, (y+1)*tileSize), graphics[c])
    return lvlImage

def getNoisedState(numpyLevel, randomSize=20):
    stringLvl=numpyToString(numpyLevel)
    state=Sokoban.State()
    state.stringInitialize(stringLvl.split("\n"))
    for i in range(randomSize):
        if random.random() < 0.5:
            state.update(0, 2*random.randint(0,1) - 1)
        else:
            state.update(2*random.randint(0,1) - 1, 0)
    return stringToNumpy(str(state))

# Dataset Classes

In [2]:
import numpy as np
import torch
from torch.utils import data

class NormalDataset(data.Dataset):
    def __init__(self, file, start=-1, end=-1):
        data = np.load(file)["input"]
        np.random.shuffle(data)
        if start <= 0:
            start = 0
        if end <= 0:
            end = len(data)
        self.data = data[start:end].astype(np.float32)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.data[idx])
    
class NoiseDataset(data.Dataset):
    def __init__(self, file, noiseLength=20, start=-1, end=-1):
        data = np.load(file)["input"]
        np.random.shuffle(data)
        if start <= 0:
            start = 0
        if end <= 0:
            end = len(data)
        self.data = data[start:end].astype(np.float32)
        self.noiseLength = noiseLength
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        noiseData = getNoisedState(self.data[idx], self.noiseLength).astype(np.float32)
        return torch.tensor(noiseData), torch.tensor(self.data[idx])

# Pytorch Model

In [12]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self, encoding=8):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(7, 128, kernel_size=(3,3), padding=2)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=(3,3), padding=1)
        self.fc1 = nn.Linear(32, encoding)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), (2,2)))
        x = F.relu(F.max_pool2d(self.conv2(x), (2,2)))
        x = F.relu(F.max_pool2d(self.conv3(x), (3,3)))
        x = x.view(-1, 32)
        x = F.relu(self.fc1(x))
        return x
    
class Decoder(nn.Module):
    def __init__(self, encoding=8):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(encoding, 32)
        self.conv1 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=1)
        self.conv3 = nn.Conv2d(128, 7, kernel_size=(3,3), padding=0)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 32, 1, 1)
        x = F.relu(self.conv1(F.interpolate(x, scale_factor=3)))
        x = F.relu(self.conv2(F.interpolate(x, scale_factor=2)))
        x = F.softmax(self.conv3(F.interpolate(x, scale_factor=2)),dim=1)
        return x
    
class AutoEncoder(nn.Module):
    def __init__(self, encoding=8):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(encoding)
        self.decoder = Decoder(encoding)
        
    def forward(self, x):
        self.bottleneck = self.encoder(x)
        return self.decoder(self.bottleneck)

# Training Code

In [19]:
import os
import matplotlib.pyplot as plt

def categorical_cross_entropy(y_pred, y_true):
    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()

def getHammingDistance(data1, data2):
    adata1=data1.argmax(axis=1)
    adata2=data2.argmax(axis=1)
    result=(adata1 != adata2).sum(dim=[1,2]).float()
    return result.mean()

def getPerfectRecostruction(data1, data2):
    adata1=data1.argmax(axis=1)
    adata2=data2.argmax(axis=1)
    result=(adata1-adata2).abs()
    result=1-result.clamp(0.0,1.0)
    result=result.sum(dim=[1,2]) == (data1.shape[2]*data1.shape[3])
    return result.sum()

def train(weightFolder, summaryWriter, logInterval, totalEpochs, network, device, trainLoader, valLoader):
    if weightFolder != None:
        os.makedirs(weightFolder)
    
    network.to(device)
    optimizer = optim.Adam(network.parameters())
    for epoch in range(totalEpochs):
        tLoss,tHamming,tCorrect,tBottleneck,tExamples = trainEpoch(network, device, optimizer, trainLoader)
        vLoss,vHamming,vCorrect,vBottleneck,vExamples = evalEpoch(network, device, valLoader)
        
        print('Epoch {}/{}:'.format(epoch+1, totalEpochs))
        print('Training:   Avg. Loss: {:.4f}, Avg. Hamming Distance: {:.4f}, Perfect Reconstruction: {}/{}'
              .format(tLoss, tHamming, tCorrect, len(trainLoader.dataset)))
        print('Validation: Avg. Loss: {:.4f}, Avg. Hamming Distance: {:.4f}, Perfect Reconstruction: {}/{}\n'
              .format(vLoss, vHamming, vCorrect, len(valLoader.dataset)))
        
        if epoch % logInterval == 0 or epoch == totalEpochs - 1:
            if summaryWriter != None:
                summaryWriter.add_scalars('Loss', {"train": tLoss, "val": vLoss}, epoch)
                summaryWriter.add_scalars('Hamming', {"train": tHamming / 100.0, "val": vHamming / 100.0}, epoch)
                summaryWriter.add_scalars('Bottleneck', {"train": tBottleneck, "val": vBottleneck}, epoch)

                trainLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in tExamples[0:4]]
                summaryWriter.add_images('train-input', trainLvls, epoch, dataformats='HWC')
                trainLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in tExamples[4:8]]
                summaryWriter.add_images('train-output', trainLvls, epoch, dataformats='HWC')
                valLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in vExamples[0:4]]
                summaryWriter.add_images('val-input', valLvls, epoch, dataformats='HWC')
                valLvls = [np.array(numpyToImage(lvl))/255.0 for lvl in vExamples[4:8]]
                summaryWriter.add_images('val-output', valLvls, epoch, dataformats='HWC')

            if weightFolder != None:
                torch.save(network.state_dict(), weightFolder + 'model_{}_{:.4f}_{:.4f}.pth'.format(epoch+1,tLoss,vLoss))
                torch.save(optimizer.state_dict(), weightFolder + 'optimizer_{}_{:.4f}_{:.4f}.pth'.format(epoch+1,tLoss,vLoss))
    
def trainEpoch(network, device, optimizer, dataLoader):
    network.train()
    trainLoss = 0
    trainHamming = 0
    trainCorrect = 0
    trainBottleneck = 0
    trainExamples = []
    for batchID, (data, target) in enumerate(dataLoader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        loss = categorical_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        trainLoss += loss.item()
        trainHamming += getHammingDistance(output, target)
        trainCorrect += getPerfectRecostruction(output, target)
        trainBottleneck += network.bottleneck.norm(dim=1).mean()
        
    for batchID, (data, target) in enumerate(dataLoader):
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        trainExamples=torch.cat([target[0:4],output[0:4]], 0).cpu().detach().numpy()
        break
    
    trainLoss /= len(dataLoader)
    trainHamming /= len(dataLoader)
    trainBottleneck /= len(dataLoader)
    
    return trainLoss, trainHamming, trainCorrect, trainBottleneck, trainExamples
    
def evalEpoch(network, device, dataLoader):
    network.eval()
    testLoss = 0
    testHamming = 0
    testCorrect = 0
    testBottleneck = 0
    testExamples = []
    with torch.no_grad():
        for batchID, (data, target) in enumerate(dataLoader):
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            testLoss += categorical_cross_entropy(output, target)
            testHamming += getHammingDistance(output, target)
            testCorrect += getPerfectRecostruction(output, target)
            testBottleneck += network.bottleneck.norm(dim=1).mean()
                
        for batchID, (data, target) in enumerate(dataLoader):
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            testExamples=torch.cat([target[0:4],output[0:4]], 0).cpu().detach().numpy()
            break
        
        testLoss /= len(dataLoader)
        testHamming /= len(dataLoader)
        testBottleneck /= len(dataLoader)
    return testLoss, testHamming, testCorrect, testBottleneck, testExamples

# Testing Network

In [ ]:
from tensorboardX import SummaryWriter
import shutil

batchSize = 128
trainSet = 15000
noiseRollout = 50
totalEpochs = 2000
logInterval = 10
experimentName = "softmax_crossentropy_loss"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

trainLoader = data.DataLoader(NormalDataset("FinalData/fixed_channel_first.npz", 0, trainSet),batch_size=batchSize, shuffle=True)
valLoader = data.DataLoader(NormalDataset("FinalData/fixed_channel_first.npz", trainSet),batch_size=batchSize, shuffle=True)

noiseTrainLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", noiseRollout, 0, trainSet),batch_size=batchSize, shuffle=True)
noiseValLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", noiseRollout, trainSet),batch_size=batchSize, shuffle=True)

trainers = {"Normal":(trainLoader, valLoader), "Noise":(noiseTrainLoader, noiseValLoader)}
encodingList = [32, 16, 8, 4]

shutil.rmtree("Experiments/Weights/{}/".format(experimentName))
shutil.rmtree("Experiments/Logs/{}/".format(experimentName))

for encoding in encodingList:
    for key, (tLoader, vLoader) in trainers.items():
        weightPath = "Experiments/Weights/{}/{}/Encoding_{}/".format(experimentName, key, encoding)
        logPath = "Experiments/Logs/{}/{}/Encoding_{}".format(experimentName, key, encoding)
        
        summaryWriter = SummaryWriter(logPath)
        autoencoder = AutoEncoder(encoding)
        train(weightPath, summaryWriter, logInterval, totalEpochs, autoencoder, device, tLoader, vLoader)

Epoch 1/2000:
Training:   Avg. Loss: 0.6884, Avg. Hamming Distance: 21.5479, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.4274, Avg. Hamming Distance: 16.6724, Perfect Reconstruction: 0/2584

Epoch 2/2000:
Training:   Avg. Loss: 0.4081, Avg. Hamming Distance: 16.2382, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.4077, Avg. Hamming Distance: 16.5765, Perfect Reconstruction: 0/2584

Epoch 3/2000:
Training:   Avg. Loss: 0.3913, Avg. Hamming Distance: 15.7113, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3885, Avg. Hamming Distance: 15.5818, Perfect Reconstruction: 0/2584

Epoch 4/2000:
Training:   Avg. Loss: 0.3797, Avg. Hamming Distance: 15.3070, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3728, Avg. Hamming Distance: 14.9560, Perfect Reconstruction: 0/2584

Epoch 5/2000:
Training:   Avg. Loss: 0.3543, Avg. Hamming Distance: 14.1009, Perfect Reconstruction: 0/15000
Validation: Avg. Loss: 0.3465, Avg. Hamming Distance: 13.7600, Perfect Reco

Epoch 42/2000:
Training:   Avg. Loss: 0.2289, Avg. Hamming Distance: 8.9536, Perfect Reconstruction: 40/15000
Validation: Avg. Loss: 0.2242, Avg. Hamming Distance: 8.7096, Perfect Reconstruction: 3/2584

Epoch 43/2000:
Training:   Avg. Loss: 0.2264, Avg. Hamming Distance: 8.8675, Perfect Reconstruction: 63/15000
Validation: Avg. Loss: 0.2272, Avg. Hamming Distance: 8.8220, Perfect Reconstruction: 5/2584

Epoch 44/2000:
Training:   Avg. Loss: 0.2259, Avg. Hamming Distance: 8.8431, Perfect Reconstruction: 39/15000
Validation: Avg. Loss: 0.2363, Avg. Hamming Distance: 9.2254, Perfect Reconstruction: 6/2584

Epoch 45/2000:
Training:   Avg. Loss: 0.2256, Avg. Hamming Distance: 8.8412, Perfect Reconstruction: 43/15000
Validation: Avg. Loss: 0.2262, Avg. Hamming Distance: 8.8312, Perfect Reconstruction: 9/2584

Epoch 46/2000:
Training:   Avg. Loss: 0.2268, Avg. Hamming Distance: 8.8828, Perfect Reconstruction: 51/15000
Validation: Avg. Loss: 0.2264, Avg. Hamming Distance: 8.8305, Perfect Reco

Epoch 82/2000:
Training:   Avg. Loss: 0.1892, Avg. Hamming Distance: 7.4826, Perfect Reconstruction: 163/15000
Validation: Avg. Loss: 0.1927, Avg. Hamming Distance: 7.5712, Perfect Reconstruction: 22/2584

Epoch 83/2000:
Training:   Avg. Loss: 0.1899, Avg. Hamming Distance: 7.5000, Perfect Reconstruction: 191/15000
Validation: Avg. Loss: 0.1989, Avg. Hamming Distance: 7.8271, Perfect Reconstruction: 21/2584

Epoch 84/2000:
Training:   Avg. Loss: 0.1881, Avg. Hamming Distance: 7.4214, Perfect Reconstruction: 180/15000
Validation: Avg. Loss: 0.1876, Avg. Hamming Distance: 7.3852, Perfect Reconstruction: 41/2584

Epoch 85/2000:
Training:   Avg. Loss: 0.1860, Avg. Hamming Distance: 7.3594, Perfect Reconstruction: 187/15000
Validation: Avg. Loss: 0.1902, Avg. Hamming Distance: 7.5006, Perfect Reconstruction: 36/2584

Epoch 86/2000:
Training:   Avg. Loss: 0.1842, Avg. Hamming Distance: 7.2887, Perfect Reconstruction: 217/15000
Validation: Avg. Loss: 0.1851, Avg. Hamming Distance: 7.2530, Per

Epoch 122/2000:
Training:   Avg. Loss: 0.1640, Avg. Hamming Distance: 6.4907, Perfect Reconstruction: 357/15000
Validation: Avg. Loss: 0.1663, Avg. Hamming Distance: 6.5336, Perfect Reconstruction: 60/2584

Epoch 123/2000:
Training:   Avg. Loss: 0.1612, Avg. Hamming Distance: 6.3649, Perfect Reconstruction: 390/15000
Validation: Avg. Loss: 0.1614, Avg. Hamming Distance: 6.3446, Perfect Reconstruction: 78/2584

Epoch 124/2000:
Training:   Avg. Loss: 0.1624, Avg. Hamming Distance: 6.4200, Perfect Reconstruction: 383/15000
Validation: Avg. Loss: 0.1730, Avg. Hamming Distance: 6.8063, Perfect Reconstruction: 44/2584

Epoch 125/2000:
Training:   Avg. Loss: 0.1620, Avg. Hamming Distance: 6.4068, Perfect Reconstruction: 339/15000
Validation: Avg. Loss: 0.1639, Avg. Hamming Distance: 6.4561, Perfect Reconstruction: 74/2584

Epoch 126/2000:
Training:   Avg. Loss: 0.1604, Avg. Hamming Distance: 6.3492, Perfect Reconstruction: 366/15000
Validation: Avg. Loss: 0.1674, Avg. Hamming Distance: 6.6172

Epoch 162/2000:
Training:   Avg. Loss: 0.1449, Avg. Hamming Distance: 5.7268, Perfect Reconstruction: 556/15000
Validation: Avg. Loss: 0.1456, Avg. Hamming Distance: 5.7204, Perfect Reconstruction: 115/2584

Epoch 163/2000:
Training:   Avg. Loss: 0.1434, Avg. Hamming Distance: 5.6664, Perfect Reconstruction: 574/15000
Validation: Avg. Loss: 0.1471, Avg. Hamming Distance: 5.7551, Perfect Reconstruction: 94/2584

Epoch 164/2000:
Training:   Avg. Loss: 0.1720, Avg. Hamming Distance: 6.6098, Perfect Reconstruction: 349/15000
Validation: Avg. Loss: 0.1708, Avg. Hamming Distance: 6.7165, Perfect Reconstruction: 31/2584

Epoch 165/2000:
Training:   Avg. Loss: 0.1455, Avg. Hamming Distance: 5.7459, Perfect Reconstruction: 502/15000
Validation: Avg. Loss: 0.1470, Avg. Hamming Distance: 5.7960, Perfect Reconstruction: 122/2584

Epoch 166/2000:
Training:   Avg. Loss: 0.1422, Avg. Hamming Distance: 5.6099, Perfect Reconstruction: 605/15000
Validation: Avg. Loss: 0.1513, Avg. Hamming Distance: 5.95

...

Epoch 954/2000:
Training:   Avg. Loss: 0.1526, Avg. Hamming Distance: 5.9150, Perfect Reconstruction: 578/15000
Validation: Avg. Loss: 0.1667, Avg. Hamming Distance: 6.2428, Perfect Reconstruction: 93/2584

Epoch 955/2000:
Training:   Avg. Loss: 0.1564, Avg. Hamming Distance: 6.0279, Perfect Reconstruction: 524/15000
Validation: Avg. Loss: 0.1685, Avg. Hamming Distance: 6.3214, Perfect Reconstruction: 81/2584

Epoch 956/2000:
Training:   Avg. Loss: 0.1512, Avg. Hamming Distance: 5.8480, Perfect Reconstruction: 574/15000
Validation: Avg. Loss: 0.1669, Avg. Hamming Distance: 6.2748, Perfect Reconstruction: 91/2584

Epoch 957/2000:
Training:   Avg. Loss: 0.1515, Avg. Hamming Distance: 5.8472, Perfect Reconstruction: 591/15000
Validation: Avg. Loss: 0.1677, Avg. Hamming Distance: 6.3849, Perfect Reconstruction: 98/2584

Epoch 958/2000:
Training:   Avg. Loss: 0.1582, Avg. Hamming Distance: 6.0785, Perfect Reconstruction: 541/15000
Validation: Avg. Loss: 0.1700, Avg. Hamming Distance: 6.3425

Epoch 994/2000:
Training:   Avg. Loss: 0.1504, Avg. Hamming Distance: 5.8011, Perfect Reconstruction: 619/15000
Validation: Avg. Loss: 0.1691, Avg. Hamming Distance: 6.3543, Perfect Reconstruction: 94/2584

Epoch 995/2000:
Training:   Avg. Loss: 0.1505, Avg. Hamming Distance: 5.8231, Perfect Reconstruction: 589/15000
Validation: Avg. Loss: 0.1652, Avg. Hamming Distance: 6.1745, Perfect Reconstruction: 99/2584

Epoch 996/2000:
Training:   Avg. Loss: 0.1517, Avg. Hamming Distance: 5.8702, Perfect Reconstruction: 582/15000
Validation: Avg. Loss: 0.1679, Avg. Hamming Distance: 6.1948, Perfect Reconstruction: 105/2584

Epoch 997/2000:
Training:   Avg. Loss: 0.1504, Avg. Hamming Distance: 5.8134, Perfect Reconstruction: 593/15000
Validation: Avg. Loss: 0.1681, Avg. Hamming Distance: 6.3031, Perfect Reconstruction: 99/2584

Epoch 998/2000:
Training:   Avg. Loss: 0.1519, Avg. Hamming Distance: 5.8777, Perfect Reconstruction: 584/15000
Validation: Avg. Loss: 0.1719, Avg. Hamming Distance: 6.410

Epoch 1034/2000:
Training:   Avg. Loss: 0.1502, Avg. Hamming Distance: 5.8034, Perfect Reconstruction: 614/15000
Validation: Avg. Loss: 0.1663, Avg. Hamming Distance: 6.2463, Perfect Reconstruction: 87/2584

Epoch 1035/2000:
Training:   Avg. Loss: 0.1494, Avg. Hamming Distance: 5.7609, Perfect Reconstruction: 593/15000
Validation: Avg. Loss: 0.1672, Avg. Hamming Distance: 6.2737, Perfect Reconstruction: 97/2584

Epoch 1036/2000:
Training:   Avg. Loss: 0.1501, Avg. Hamming Distance: 5.7974, Perfect Reconstruction: 582/15000
Validation: Avg. Loss: 0.1663, Avg. Hamming Distance: 6.1767, Perfect Reconstruction: 100/2584

Epoch 1037/2000:
Training:   Avg. Loss: 0.1501, Avg. Hamming Distance: 5.7924, Perfect Reconstruction: 630/15000
Validation: Avg. Loss: 0.1697, Avg. Hamming Distance: 6.3264, Perfect Reconstruction: 94/2584

Epoch 1038/2000:
Training:   Avg. Loss: 0.1512, Avg. Hamming Distance: 5.8460, Perfect Reconstruction: 587/15000
Validation: Avg. Loss: 0.1692, Avg. Hamming Distance: 

Epoch 1074/2000:
Training:   Avg. Loss: 0.1507, Avg. Hamming Distance: 5.8167, Perfect Reconstruction: 559/15000
Validation: Avg. Loss: 0.1789, Avg. Hamming Distance: 6.6360, Perfect Reconstruction: 81/2584

Epoch 1075/2000:
Training:   Avg. Loss: 0.1566, Avg. Hamming Distance: 6.0356, Perfect Reconstruction: 521/15000
Validation: Avg. Loss: 0.1682, Avg. Hamming Distance: 6.2793, Perfect Reconstruction: 107/2584

Epoch 1076/2000:
Training:   Avg. Loss: 0.1538, Avg. Hamming Distance: 5.9031, Perfect Reconstruction: 556/15000
Validation: Avg. Loss: 0.1998, Avg. Hamming Distance: 6.9998, Perfect Reconstruction: 55/2584

Epoch 1077/2000:
Training:   Avg. Loss: 0.1525, Avg. Hamming Distance: 5.8912, Perfect Reconstruction: 560/15000
Validation: Avg. Loss: 0.1665, Avg. Hamming Distance: 6.2204, Perfect Reconstruction: 98/2584

Epoch 1078/2000:
Training:   Avg. Loss: 0.1481, Avg. Hamming Distance: 5.7117, Perfect Reconstruction: 624/15000
Validation: Avg. Loss: 0.1691, Avg. Hamming Distance: 

Epoch 1114/2000:
Training:   Avg. Loss: 0.1579, Avg. Hamming Distance: 6.0607, Perfect Reconstruction: 500/15000
Validation: Avg. Loss: 0.1786, Avg. Hamming Distance: 6.5987, Perfect Reconstruction: 67/2584

Epoch 1115/2000:
Training:   Avg. Loss: 0.1495, Avg. Hamming Distance: 5.7823, Perfect Reconstruction: 600/15000
Validation: Avg. Loss: 0.1661, Avg. Hamming Distance: 6.1190, Perfect Reconstruction: 106/2584

Epoch 1116/2000:
Training:   Avg. Loss: 0.1472, Avg. Hamming Distance: 5.6892, Perfect Reconstruction: 662/15000
Validation: Avg. Loss: 0.1673, Avg. Hamming Distance: 6.1731, Perfect Reconstruction: 84/2584

Epoch 1117/2000:
Training:   Avg. Loss: 0.1476, Avg. Hamming Distance: 5.6858, Perfect Reconstruction: 610/15000
Validation: Avg. Loss: 0.1707, Avg. Hamming Distance: 6.3072, Perfect Reconstruction: 90/2584

Epoch 1118/2000:
Training:   Avg. Loss: 0.1489, Avg. Hamming Distance: 5.7577, Perfect Reconstruction: 609/15000
Validation: Avg. Loss: 0.1772, Avg. Hamming Distance: 

# Data Testing

In [ ]:
import matplotlib.pyplot as plt

noiseTrainLoader = data.DataLoader(NoiseDataset("FinalData/fixed_channel_first.npz", 50, 0, 15000),batch_size=128, shuffle=True)

for idx, (miniInput,miniOutput) in enumerate(noiseTrainLoader):
    if idx == 0:
        result = torch.cat([miniInput[0:4],miniOutput[0:4]],0)
        plt.figure()
        plt.imshow(numpyToImage(result[0]))
        plt.show()
        
        plt.figure()
        plt.imshow(numpyToImage(result[4]))
        plt.show()

# Fix the Dataset

In [ ]:
levels = np.load('FinalData/channel_first_data.npz')

directions = {(-1,0), (1,0), (0,1), (0,-1)}

def floodFill(lvl, x, y, label):
    result = []
    queue = [(x,y)]
    while(len(queue) > 0):
        current = queue.pop(0)
        if lvl[current[1]][current[0]] == 0:
            lvl[current[1]][current[0]] = label
            result.append(current)
            for d in directions:
                new = (current[0]+d[0], current[1]+d[1])
                if new[0] < 0 or new[1] < 0 or new[0] >= lvl.shape[1] or new[1] >= lvl.shape[0]:
                    continue
                queue.append(new)
    return result

def floodFillCheck(lvl):
    finalResult=[]
    tempLvl = np.zeros((10,10))
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            if lvl[y][x] == 0:
                tempLvl[y][x] = -1
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            if tempLvl[y][x] == 0:
                finalResult.append(floodFill(tempLvl,x,y,len(finalResult)+1))
    return finalResult

def fixLevel(lvl, regions):
    fixed = np.zeros(lvl.shape)
    for y in range(lvl.shape[0]):
        for x in range(lvl.shape[1]):
            fixed[y][x] = lvl[y][x]
    maxRegion = []
    for r in regions:
        if len(r) > len(maxRegion):
            maxRegion = r
    for r in regions:
        if r != maxRegion:
            for p in r:
                fixed[p[1]][p[0]] = 0
    finalLvl = np.zeros((7,10,10))
    for y in range(fixed.shape[0]):
        for x in range(fixed.shape[1]):
            finalLvl[int(fixed[y][x])][y][x] = 1
    return finalLvl

fixedLevels = []
total = 0
for lvl in levels["input"]:
    total += 1
    regions = floodFillCheck(np.argmax(lvl,axis=0))
    if len(regions) > 1:
        fixedLevels.append(fixLevel(np.argmax(lvl,axis=0), regions))
    else:
        fixedLevels.append(lvl)

np.savez('FinalData/fixed_channel_first.npz', input=fixedLevels, output=fixedLevels)